In [209]:
file = open("testcase_included_by_us_2.cpp","r")

In [210]:
inp = file.read()

In [211]:
inp

'#include <iostream>\nusing namespace std;\nint main() {\n int my_int = 10; // No syntax error here\n cout << "Integer: " << my_int << endl;\n return 0;\n}\n'

In [212]:
def removeComments(inp):
    parsed = ""
    si = len(inp)
    
    sing_comm = False
    mul_comm = False
    i = 0
    while i <si:
        ch = inp[i]
        if sing_comm:
            if ch == '\n':
                sing_comm = False                
        elif mul_comm:
            if ch == '*' and (i+1<si and inp[i+1]=='/'):
                mul_comm = False
                i+=1
                
        elif ch == '/':
            if i+1<si and inp[i+1]=='/':
                sing_comm = True
            elif i+1<si and inp[i+1]=='*':
                mul_comm = True
            else:
                parsed+=ch
        else:
            parsed+=ch
        i+=1
    return parsed

In [213]:
inp = removeComments(inp)

In [214]:
def checkOperator(c):
    ops = [',',';',':','?','[',']','{','}','(',')','+','=','-','|','&','^','*','/','!','<','>']
    return c in ops

In [215]:
def checkEqual(c):
    ops = ['+','-','*','/','&','|','^','!','<','>', '=']
    return c in ops

In [216]:
def checkSame(c):
    ops = ['+','-','&','|','<','>']
    return c in ops

In [217]:
def checkBrackets(c):
    ops = ['[',']','{','}','(',')']
    return c in ops

In [218]:
tokens = []

temp_str = ""
si = len(inp)
i = 0
const_str = False
char_lit = False
line_number = 1  # Track line number

while i < si:
    ch = inp[i]
    
    if ch == '\n':
        line_number += 1  # Increment line number
        if const_str or char_lit:
            #print("Error: Unclosed quote at line", line_number)
            break
        i += 1  # Move to next character
        continue
    
    i += 1  # Move to next character
    
    if ch == '"':
        temp_str += ch
        if const_str:
            const_str = False
        else:
            const_str = True
        continue
    
    if ch == '\'':
        temp_str += ch
        if char_lit:
            char_lit = False
        else:
            char_lit = True
        continue
    
    if const_str or char_lit:
        temp_str += ch
        continue
    
    if ch == ' ' or ch == '\t':
        if temp_str != "":
            tokens.append(temp_str)
            temp_str = ""
    elif checkOperator(ch):
        if temp_str != "":
            tokens.append(temp_str)
            temp_str = ""
        
        if checkBrackets(ch):
            tokens.append(ch)
            continue
            
        if ch == '-':
            if i < si and inp[i] == '>':
                tokens.append("->")
                i += 1
                continue
        
        if checkSame(ch):
            if i < si and inp[i] == ch:
                tokens.append("" + ch + ch)
                i += 1
                continue
                
        if checkEqual(ch):
            if i < si and inp[i] == '=':
                tokens.append("" + ch + "=")
                i += 1
                continue
        
        tokens.append(ch)
    else:
        temp_str += ch

# Check for unclosed quotes at the end of the input

In [219]:
tokens

['#include',
 '<',
 'iostream',
 '>',
 'using',
 'namespace',
 'std',
 ';',
 'int',
 'main',
 '(',
 ')',
 '{',
 'int',
 'my_int',
 '=',
 '10',
 ';',
 'cout',
 '<<',
 '"Integer: "',
 '<<',
 'my_int',
 '<<',
 'endl',
 ';',
 'return',
 '0',
 ';',
 '}']

In [220]:
# !pip install pickleshare

In [221]:
# %store tokens
import pickleshare

# Create a PickleShare database
db = pickleshare.PickleShareDB('~/.ipython/profile_default/db')

# Store variables
db['tokens'] = tokens
db['char_lit'] = char_lit
db['const_str'] = const_str
